
##Simulation Settings

In [ ]:
!pip install torch==1.13.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.1/890.1 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.3 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.3.1+cu121
    Uninstalling torch-2.3.1+cu121:
      Successfully uninstalled torch-2.3.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.1+cu121 requires torch==2.3.1, but you have torch 1.13.0 which is incompatible.
torchtext 0.18.0 requires torch>=2.3.0, but you have torch 1.13.0 which is incompatible.
torchvision 0.18.1+cu121 requires torch==2.3.1, but you have torch 1.13.0 

##Fast Train Unfolding

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torch.linalg import svd, eig
import math
import time
import scipy.io as sio

torch.set_printoptions(precision=5, sci_mode=False)



def set_params(n_0= 1, p_s= 1, B= 2, K= 7, P= 4, N= 2,
              L= 2, T= 5, epochs= 500, J= 10,
              W_is_phase_only= True, W_is_block_diag= True,
              batch_size= 50, pga_iters= 500, lr_mu_beta= 0.0001,
              lr_lambda= 0.001, binary_thresh= 0.5,
              mat_file_path= "/content/H_1000_2_8_7.mat", seed= 1,
              train_size= 750, valid_size= 250, lambdas_init= 0.0,
              gamma= 200 , project_every= 1, l1_target = 1.0, sharpness = 30,
              Q = 4, W_is_finite = False):
    """
    This function sets the general parameters of the algorithm.
    Used in order to save space in the code when running multiple simulations.
    All of these parameters are predifined and can be modified.
    :param n_0: noise power
    :param p_s: signal power
    :param B: number of frequency bins
    :param K: number of users to be served
    :param P: number of panels in the base station
    :param N: number of antennas in each panel
    :param L: number of outputs in each panel
    :param T: number of inputs to the CPU
    :param epochs: number of epochs in the training
    :param J: number of iteration to be unfolded , usually 10.
    :param batch_size: batch size in the training part.
    :param pga_iters: number of iterations in the PGA algorithm to be compared with.
    :param lr_mu_beta: learning rate of the optimizer. denoted as eta in the paper.
    :param lr_lambda: learning rate of the optimizer for optimzing lambda.
    :param binary_thresh: the threshold for setting elements to zero in the projection onto phases.
    :param train_size: size of the channel realizations taken to training set.
    :param valid_size: size of the channel realizations taken to validation set.
    :return: all the parameters.
    """

    # General parameters
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    M = P * N  # Total number of antennas in the Base station
    Ik = torch.eye(T, dtype=torch.complex128, device = device)
    Im = torch.eye(M, dtype=torch.complex128, device = device)
    snr = p_s / n_0

    # Building the A matrix using the examples given in the paper
    # "Trade-Offs in Decentralized Multi-Antenna Architectures: The WAX Decomposition"
    A = torch.empty((8,5))
    A[:5, :] = torch.eye(5)
    A[5:, :3] = torch.eye(3)
    A[5:, 3:] = 1
    A = A.to(dtype=torch.complex128, device = device)

    if T == 5*2:
        A = torch.block_diag(A,A)

    if T == 5*8:
        A = torch.block_diag(A,A,A,A,A,A,A,A)

    if T == 5*16:
        A = torch.block_diag(A,A,A,A,A,A,A,A)
        A = torch.block_diag(A,A)

    if T == 61: # suitable only for the case where L*P = 100

        A = torch.empty((100, 61))
        A[ :61, :] = torch.eye(61)
        A[61:, :39] = torch.eye(39)
        A[61:83, 39:] = torch.eye(22)
        A[83:, 39:56] = torch.eye(17)
        A[83:88, 56:] = torch.eye(5)
        A[88:93, 56:] = torch.eye(5)
        A[93:98, 56:] = torch.eye(5)
        A[98:, 56:58] = torch.eye(2)
        A[98:, 58:60] = torch.eye(2)
        A = A.to(dtype=torch.complex128, device = device)

    # Creating D to be a mask matrix that will be used to project the matrix W onto a block diagonal matrix.
    D = torch.zeros((M, L * P), device = device)
    for p in range(P):
        p_n, p_l = p * N, p * L
        D[p_n:(p_n + N), p_l:(p_l + L)] = torch.ones((N, L))

    # Creating W:
    torch.manual_seed(seed)
    rand_phases = torch.rand(N * L * P, device=device)
    W_vec = torch.exp(1j * 2*math.pi * rand_phases).to(dtype=torch.complex128)
    W = torch.block_diag(*W_vec.view(P, N, L))
    return W, B, K, P, N, M, L, T, snr, A, Ik, Im, D, J, epochs, W_is_block_diag, W_is_phase_only, batch_size, pga_iters, lr_mu_beta, lr_lambda, binary_thresh, train_size, valid_size, lambdas_init, gamma , mat_file_path, project_every, l1_target, device, sharpness, Q, W_is_finite


def soft_mag_proj(w, binary_thresh, sharpness):
    w = torch.exp(1j*torch.angle(w)) / (1 + torch.exp(-sharpness * (torch.abs(w) - binary_thresh)))
    return w


def hard_mag_proj(w, binary_thresh):
    w = D * torch.exp(1j*torch.angle(w)) * torch.where(torch.abs(w) < binary_thresh, torch.zeros_like(w), torch.ones_like(w))
    return w


def soft_phase_proj(W, sharpness):
    output_phase = torch.zeros_like(W)
    for q in range(1, Q + 1):
        # Calculate the shifted and normalized sigmoid for each q
        sigmoid_output_q = 2*np.pi/(1+torch.exp(-sharpness*(W.angle()-(q-0.5)*2*torch.pi/Q + np.pi))) - np.pi
        output_phase += sigmoid_output_q / Q
    return output_phase


def hard_phase_proj(W):
    output_phase = torch.zeros_like(W)
    for q in range(0, Q+1):
        # Calculate the shifted and normalized step function for each q
        step_output_q = torch.where((W.angle() >= (q-0.5)*2*np.pi/Q - np.pi) & (W.angle() < (q+0.5)*2*np.pi/Q - np.pi), q*2*np.pi/Q - np.pi, 0.0)
        output_phase += step_output_q
    return output_phase


class FastTrainUnfolding(nn.Module):
    def __init__(self, W, J, forward_path='Unfolding', mu = None, beta = None, cnn_layers = 10):
        super(FastTrainUnfolding, self).__init__()
        self.J = J
        self.W = W.detach().clone()  # Make a copy to avoid modifying the original W
        self.ones = torch.ones(1, dtype = torch.float64, device = device)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.pga_iters = pga_iters
        self.constant_mu = mu
        self.constant_beta = beta
        self.cnn_layers = cnn_layers
        self.PE = project_every


        # Define layers for Unfolded:
        self.Mu = nn.ModuleList([nn.Linear(1, P*N*L, dtype = torch.float64) for _ in range(J)])
        self.Beta = nn.ModuleList([nn.Linear(1, P*N*L, dtype = torch.float64) for _ in range(J)])
        self.Lambda = nn.ModuleList([nn.Linear(1, P*N*L, dtype = torch.float64) for _ in range(J)])

        if forward_path == "CNN":
            # Define layers for the CNN.
            mag_layers = []
            for _ in range(self.cnn_layers - 1):
              mag_layers.extend([nn.Conv2d(B, B, kernel_size = 4, padding = 2, stride = 1, dtype = torch.float64),
                                nn.MaxPool2d(kernel_size = 2, stride = 2),
                                nn.Flatten(),
                                nn.Linear(((M+1)//2 * ((K+1)//2) * B ), B*M*K, dtype = torch.float64),
                                nn.Unflatten(-1,(B,M,K)),
                                nn.BatchNorm2d(B, dtype = torch.float64),
                                ])
            self.cnn_mag = nn.Sequential(*mag_layers)

            phase_layers = []
            for _ in range(self.cnn_layers - 1):
              phase_layers.extend([nn.Conv2d(B, B, kernel_size = 4, padding = 2, stride = 1, dtype = torch.float64),
                                nn.BatchNorm2d(B, dtype = torch.float64),
                                nn.MaxPool2d(kernel_size = 2, stride = 2),
                                nn.Flatten(),
                                nn.Linear(((M+1)//2 * ((K+1)//2) * B), B*M*K, dtype = torch.float64),
                                nn.Unflatten(-1,(B,M,K))
                                ])
            self.cnn_phase = nn.Sequential(*phase_layers)


            self.cnn_f_layer = nn.Linear(2 * B * M * K, N * P * L, dtype = torch.float64)

        # Define the selected forward path
        if forward_path == 'Unfolding':
            self.forward_path = self.forward_unfolded
        elif forward_path == 'Constant':
            self.forward_path = self.forward_constant
        elif forward_path == "CNN":
            self.forward_path = self.forward_cnn
        else:
            raise ValueError("Invalid forward_path. Choose 'Unfolding' or 'Constant'.")


    def forward(self, H):
        return self.forward_path(H)


    def forward_cnn(self, H):
        # Convert complex number to magnitude and phase
        H_mag, H_phase = torch.abs(H), torch.angle(H)

        # Pass through the neural network layers
        output_mag = self.cnn_mag(H_mag).view(H_mag.size(0), -1)
        output_phase = self.cnn_phase(H_phase).view(H_phase.size(0), -1)
        final_vec = torch.cat((output_mag, output_phase), dim=1)

        # Convert magnitude and phase back to complex number
        W_phases_vec = self.cnn_f_layer(final_vec)
        # if W_is_finite:
        #     W_phases_vec = hard_phase_proj(W_phases_vec)
        W_phases_blocks = W_phases_vec.view(-1,P,N,L)
        W_pred = D * torch.exp(1j * 2*math.pi * vec_2_blk_diag(W_phases_vec, batch=True))
        cnn_R = self.calc_R(H,W_pred)
        return W_pred, cnn_R


    def forward_unfolded(self, H):
        W_1 = self.W.clone()
        Rs = torch.zeros(H.shape[0], J)
        l1_norms = torch.zeros(H.shape[0], J)
        W_0 = torch.zeros_like(W_1)
        for j in range(self.J):
            grad = self.calc_dR_dW(H, W_1)
            weights = self.Mu[j](self.ones)
            Mu_step = vec_2_blk_diag(weights) * grad
            Momentum = vec_2_blk_diag(self.Beta[j](self.ones)) * (W_1 - W_0)
            norm_reduc = 0.001*vec_2_blk_diag(self.Lambda[j](self.ones)) * torch.exp(1j*torch.angle(W_1))
            ## Updating:
            W_2 = W_1 + Mu_step + Momentum  - norm_reduc

            ## Projecting:
            ## If training, use Soft Projecting:
            if (j+1) % self.PE == 0 and self.training:
                W_2 = D * soft_mag_proj(W_2, binary_thresh = binary_thresh, sharpness = sharpness)
                if W_is_finite:
                    W_2 = D * torch.exp(1j*soft_phase_proj(W_2, sharpness = sharpness))

            ## If Validating, use Hard Projecting:
            if (j+1) % self.PE == 0 and not self.training:
                W_2 = hard_mag_proj(W_2, binary_thresh = binary_thresh)
                if W_is_finite:
                    W_2 = D * torch.exp(1j*hard_phase_proj(W_2))


            ## Documenting:
            l1_norms[:,j] = torch.norm(W_2, p=1)
            Rs[:,j] = self.calc_R(H, W_2)
            W_0 = W_1
            W_1 = W_2

        return W_2, Rs, l1_norms


    def forward_constant(self, H):
        W_1 = self.W.clone()
        W_0 = torch.zeros_like(W_1)
        Rs = torch.zeros(H.shape[0], self.pga_iters)
        for iter in range(self.pga_iters):
            grad = self.calc_dR_dW(H, W_1)
            Mu_step = self.constant_mu * grad
            Momentum = self.constant_beta * (W_1 - W_0)
            ## Updating:
            W_2 = W_1 + Mu_step + Momentum
            ## Projecting:
            if (iter+1) % self.PE == 0:
                W_2 = D * (torch.exp(1j * torch.angle(W_2)))
                if W_is_finite:
                    W_2 = D * torch.exp(1j*hard_phase_proj(W_2))

            ## Documenting:
            Rs[:,iter] = self.calc_R(H, W_2)
            W_0 = W_1
            W_1 = W_2
        return W_2, Rs



    def calc_dR_dW(self, H, W):
        """calculates the gradient of the rate with respect to W. the gradient has the same dimension as W"""
        grad = torch.zeros(H.size(0), M, L*P, dtype = torch.complex128, device =self.device)
        for b in range(B):
            h1 = H[:,b, :, :]
            W_con = W.conj()
            H_con = h1.conj()
            # print(W.shape)
            # print(A.shape)
            G = W @ A
            if len(G.shape) < 3:
              G = G.unsqueeze(0).repeat(H.shape[0], 1, 1)
            G_herm = G.transpose(1, 2).conj()
            h1_herm = h1.transpose(1, 2).conj()
            G_psu_inv = torch.inverse(G_herm @ G + 10**-6 * torch.eye(T, device = self.device))
            Proj_mat = G @ G_psu_inv @ G_herm
            Z = snr * G_psu_inv @ G_herm @ h1 @ h1_herm @ G
            dR_dZ = torch.inverse(Ik + Z)
            diff = snr * (A @ dR_dZ @ (2 * G_psu_inv @ G_herm) @ h1 @ h1_herm @ (Im - Proj_mat)).conj().transpose(1,2)
            grad += diff
        return grad / B


    def calc_dR_dW2(self, H, W):
        """calculates the gradient of the rate with respect to W. the gradient has the same dimension as W"""
        grad = torch.zeros(H.size(0), M, L*P, dtype = torch.complex128, device =self.device)
        for b in range(B):
            h1 = H[:,b, :, :]
            W_con = W.conj()
            H_con = h1.conj()
            # print(W.shape)
            # print(A.shape)
            G = W @ A
            if len(G.shape) < 3:
              G = G.unsqueeze(0).repeat(H.shape[0], 1, 1)
            G_herm = G.transpose(1, 2).conj()
            h1_herm = h1.transpose(1, 2).conj()
            G_psu_inv = torch.inverse(G_herm @ G + 10**-10 * torch.eye(T, device = self.device)) @ G_herm
            Proj_mat = G @ G_psu_inv
            Z = snr * G_psu_inv @ h1 @ h1_herm @ G
            dR_dZ = torch.inverse(Ik + Z)
            diff = 2 * snr * (A @ dR_dZ @ G_psu_inv  @ h1 @ h1_herm @ (Im - Proj_mat)).conj().transpose(1,2)
            grad += diff
        return grad / B


    def calc_dR_dW3(self, H, W):
            """calculates the gradient of the rate with respect to W. the gradient has the same dimension as W"""
            grad = torch.zeros(H.size(0), M, L*P, dtype = torch.complex128, device =self.device)
            for b in range(B):
                h1 = H[:,b, :, :]
                W_con = W.conj()
                H_con = h1.conj()
                G = W @ A
                if len(G.shape) < 3:
                  G = G.unsqueeze(0).repeat(H.shape[0], 1, 1)
                G_herm = G.transpose(1, 2).conj()
                h1_herm = h1.transpose(1, 2).conj()
                G_psu_inv = torch.linalg.inv(G_herm @ G + 10**-10 * torch.eye(T, device = self.device)) @ G_herm
                Proj_mat = G @ G_psu_inv
                G_pinv_X_h_X_h_herm = G_psu_inv @ h1 @ h1_herm
                Z = snr * G_pinv_X_h_X_h_herm @ G
                dR_dZ = torch.linalg.inv(Ik + Z)
                diff = 2 * snr * (A @ dR_dZ @ G_pinv_X_h_X_h_herm @ (Im - Proj_mat)).conj().transpose(1,2)
                grad += diff
            return grad / B





    def calc_R(self, H, W):
        R = 0
        G = W @ A
        G_herm = G.transpose(1, 2).conj()
        for b in range(B):
          h1 = H[:,b,:,:]
          h1_herm = h1.transpose(1, 2).conj()
          psudo_inv_G = torch.inverse(G_herm @ G + 10**-6 * torch.eye(T,device = self.device))
          proj_mat = G @ psudo_inv_G @ G_herm
          R += torch.log((Im + snr * proj_mat @ h1 @ h1_herm).det())
        return R/B


    def double_weights(self):
        # Double the weights
        for module_list in [self.Mu, self.Beta, self.Lambda]:
            for module in module_list:
                new_weights = torch.cat((module.weight, module.weight.clone()), dim=0).detach()
                module.weight = torch.nn.Parameter(new_weights)

                 # Double bias (if bias exists)
                if module.bias is not None:
                    new_bias = torch.cat((module.bias, module.bias.clone()), dim=0).detach()
                    module.bias = torch.nn.Parameter(new_bias)



class QuaDRiGA(Dataset):
    def __init__(self):
        self.num_samples = valid_size + train_size
        self.H = sio.loadmat(mat_file_path)['H']
        self.H = torch.tensor(self.H.transpose(0, 3, 2, 1)).to(device)

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        sample = self.H[idx,:B,:,:]
        return sample


class IID(Dataset):
    def __init__(self):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.num_samples = valid_size + train_size
        self.H = torch.randn((self.num_samples,B,M,K), dtype = torch.complex128, device = device)

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        sample = self.H[idx,:,:,:]
        return sample


def vec_2_blk_diag(vec, batch=False):
    if batch:
        size_batch = vec.shape[0]
        assert len(vec.shape) == 2, "Invalid shape for batched vector"
        assert vec.shape[1] == P * N * L, "Invalid vector length for batched vector"
        blocks = vec.view(size_batch, P, N, L)
    else:
        assert len(vec.shape) == 1, "Invalid shape for non-batched vector"
        assert vec.shape[0] == P * N * L, "Invalid vector length for non-batched vector"
        blocks = vec.view(P, N, L)

    if batch:
        result_matrix = torch.cat([torch.block_diag(*blocks[i].unbind(0)).unsqueeze(0) for i in range(size_batch)], dim=0)
    else:
        result_matrix = torch.block_diag(*blocks.unbind(0)).unsqueeze(0)

    return result_matrix


def train_and_plot(model, num_epochs, train_loader, val_loader, cnn = False, seq_learning = False, pre_model= None, plot = True):
    train_losses = []
    val_losses = []
    l1_norms_list_train = []
    l1_norms_list_val = []
    log_weights = torch.log(torch.arange(2, J + 2))

    # Training
    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0.0
        total_l1_norms = 0.0
        for H_batch in train_loader:
            if not cnn: # If we are training the Unfolded network
                u_optimizer.zero_grad()
                lam_optimizer.zero_grad()
                if seq_learning:
                    W_out_pre,Rs_pre = pre_model(H_batch)
                    model.W = W_out_pre
                W_out_u, Rs, l1_norms = model(H_batch)
                l1_norms = l1_norms.mean(dim=0) / (N*P*L*(Rs.shape[0])) # averaging on them
                Rs = Rs.mean(dim = 0) # averaging on them

                loss = -torch.sum(Rs * log_weights) # Taking the last value of R for each sample
                loss += gamma * (torch.sum(log_weights*(l1_norms - l1_target)**2))
                loss.backward()
                u_optimizer.step()
                lam_optimizer.step()

                total_train_loss += Rs[-1].item() / K
                total_l1_norms += l1_norms[-1].item()


            else: # If we are training the CNN network
                cnn_optimizer.zero_grad()
                W_out_cnn, cnn_R = model(H_batch)
                if seq_learning: # Learning a sequence of networks
                    pre_model.W = W_out_cnn
                    W_out_post, Rs_post, _ = pre_model(H_batch)
                    loss = -torch.sum(Rs_post.mean(dim=0) * log_weights) # Taking the last value of R for each sample
                    loss.backward()
                    cnn_optimizer.step()
                    total_train_loss += Rs_post.mean(dim=0)[-1].item() / K

                else: # Regular Learning of CNN
                    cnn_R = cnn_R.mean(dim = 0) # averaging on them
                    loss = -cnn_R
                    loss.backward()
                    cnn_optimizer.step()
                    total_train_loss += cnn_R.item() / K


        average_train_loss = total_train_loss / len(train_loader)
        train_losses.append(average_train_loss)
        if not cnn:
          l1_norms_list_train.append(total_l1_norms / len(train_loader))

        # Validation
        model.eval()
        with torch.no_grad():
            total_val_loss = 0.0
            val_rate = 0.0
            total_l1_norms_v = 0.0
            for H_val_batch in val_loader:
                if not cnn: # If we are training the Unfolded network
                    if seq_learning:
                        W_out_pre,Rs_pre = pre_model(H_val_batch)
                        model.W = W_out_pre
                    W_val_pred, Rs_val, l1_norms_v = model(H_val_batch)
                    l1_norms_v = l1_norms_v.mean(dim=0) / (P*N*L*(Rs_val.shape[0]))
                    Rs_val = Rs_val.mean(dim=0) / K
                    val_rate += Rs_val
                    total_val_loss += Rs_val[-1].item()
                    total_l1_norms_v += l1_norms_v[-1].item()
                else: # If we are training the CNN network
                    W_val_pred, cnn_R_val = model(H_val_batch)
                    if seq_learning:
                        pre_model.W = W_val_pred
                        W_val_post,Rs_post,_ = pre_model(H_val_batch)
                        cnn_R_val = Rs_post.mean(dim=0)[-1] / K
                    else:
                        cnn_R_val = cnn_R_val.mean(dim=0) / K
                    val_rate += cnn_R_val
                    total_val_loss += cnn_R_val.item()

            average_val_loss = total_val_loss / len(val_loader)
            val_rate = val_rate / len(val_loader)
            val_losses.append(average_val_loss)
            if not cnn:
                l1_norms_list_val.append(total_l1_norms_v / len(val_loader))



        # Print the loss for each epoch
        if (epoch +1) % 50 == 0:
          print(f'Epoch [{epoch + 1}/{num_epochs}], '
                f'Training: {round(abs(average_train_loss), 5)}, '
                f'Validation: {round(abs(average_val_loss), 5)}')

    if plot:
        # Plot the training and validation losses
        plt.plot(train_losses, label='Train')
        plt.plot(val_losses, label='Valid')
        plt.xlabel('Epochs')
        plt.ylabel('Channel Rate')
        plt.title('CNN Training' if cnn else 'Unfolding Training')
        plt.legend()
        plt.grid()
        plt.show()
    return val_rate, model, l1_norms_list_val, l1_norms_list_train


def constant_mu_and_beta_search():
  mu_grid = torch.cat((torch.linspace(0.01,0.5,6),torch.linspace(1,5,5)))
  beta_grid = torch.cat((torch.linspace(0,0.99,3),torch.tensor([0.9])))
  search_Rs = 0
  best_Rs = torch.zeros(pga_iters)
  best_mu = 0
  best_beta = 0
  indices = np.arange(project_every, pga_iters +1, project_every)
  for mu in mu_grid:
      for beta in beta_grid:
        con_model = FastTrainUnfolding(W, J, "Constant", mu, beta)
        search_Rs = 0
        for H_val_batch in val_loader:
          W_val_pred, Rs_val = con_model(H_val_batch)
          Rs_val = Rs_val.mean(dim=0) / K
          search_Rs += Rs_val
        search_Rs = search_Rs / len(val_loader)
        if max(abs(search_Rs[indices-1])) > max(abs(best_Rs[indices-1])):
          best_Rs = search_Rs
          best_mu = mu
          best_beta = beta
          # print(f' Updated, mu: {round(mu.item(),2)}, beta: {round(beta.item(),2)}, R: {round(max(abs(best_Rs)).item(),6)}')

  print(f'best mu: {round(best_mu.item(),2)}, best beta: {round(best_beta.item(),2)}, best R: {round(max(abs(best_Rs)).item(),6)}')
  return best_Rs, best_mu, best_beta


def plot_R(unfolding_R, best_Rs, ls_Rs, cnn_R, Mo_Rs, alt_learn_Rs= None, full=False):
    indices = np.arange(project_every,len(best_Rs)+1, project_every)
    u_indices = np.arange(project_every,len(unfolding_R)+1,project_every)
    ls_indices = np.arange(project_every,len(ls_Rs)+1, project_every)
    mo_indices = np.arange(project_every,len(Mo_Rs)+1, project_every)

    plt.plot(u_indices, unfolding_R[u_indices-1],"-o", label='U-PGA+M', color = "blue")
    plt.plot(indices, best_Rs.detach().numpy()[indices-1],"--^", label = "PGA+M", color = "orange")
    plt.plot(ls_indices, ls_Rs.detach().numpy()[ls_indices-1],"--*", label = "Line Search", color = "green")
    plt.plot(mo_indices, Mo_Rs.detach().numpy()[mo_indices-1],"-.", label = "MO", color = "cyan")
    if alt_learn_Rs is not None:
          alt_Rs_indices = np.arange(project_every,len(alt_learn_Rs)+1, project_every)
          plt.plot(alt_Rs_indices, alt_learn_Rs.detach().numpy()[alt_Rs_indices-1],"-x", label = "CNN + U-PGA+M", color = "red")

    plt.axhline(y=abs(cnn_R.item()), color='magenta', linestyle=':', label="CNN - {} layers".format(cnn_model.cnn_layers))
    plt.axhline(y=max(abs(best_Rs[indices-1].detach().numpy())), color='black', linestyle='--', label='PGA+M - {} iters'.format(pga_iters))

    plt.xlabel('Iteration')
    plt.ylabel("Bits per channel use per user")
    plt.legend()
    plt.grid()
    if not full:
        plt.xlim(project_every,J)
    plt.show()


def plot_norms():
    plt.plot(range(1,len(u_l1_norms_v)+1),u_l1_norms_v, label="Valid")
    plt.plot(range(1,len(u_l1_norms_t)+1),u_l1_norms_t, label= "Train")
    plt.ylabel('% of components on')
    plt.xlabel("epochs")
    plt.axhline(y=l1_target, color='k', linestyle='--', label = "target")
    plt.grid()
    plt.title("L1 Norms")
    plt.legend()
    plt.show()


def gen_data(train_size,valid_size,dataset="QuaDRiGA"):
  if dataset == "QuaDRiGA":
    dataset = QuaDRiGA()

  if dataset == "I.I.D.":
      dataset = IID()

  # # Split the dataset into training and validation sets
  train_dataset = torch.utils.data.Subset(dataset, range(valid_size, train_size + valid_size))
  val_dataset = torch.utils.data.Subset(dataset, range(valid_size))

  # # Create DataLoaders for training and validation
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
  val_loader_singles = DataLoader(val_dataset, batch_size=1, shuffle=True)
  return train_loader, val_loader, val_loader_singles


def show_mu_and_beta(model):

    # Create subplots for Mu
    fig, axs_mu = plt.subplots(2, (J + 1) // 2, figsize=(21, 8.5))

    # Calculate global min and max for Mu
    global_min_mu = min([min(model.Mu[j](model.ones)) for j in range(J)])
    global_max_mu = max([max(model.Mu[j](model.ones)) for j in range(J)])

    # After training is complete
    for j, linear_layer in enumerate(model.Mu):
        # Pass ones vector through the linear layer, to see its values
        Mu_j = vec_2_blk_diag(model.Mu[j](model.ones))


        # Plot magnitude
        im1 = axs_mu[j // ((J + 1) // 2), j % ((J + 1) // 2)].imshow(Mu_j.squeeze(0).cpu().detach().numpy(), cmap='viridis', vmin=global_min_mu, vmax=global_max_mu)
        axs_mu[j // ((J + 1) // 2), j % ((J + 1) // 2)].set_title(f'Layer {j}')

    # Create a colorbar for Mu
    cbar_mu_ax = fig.add_axes([0.25, 0.08, 0.5, 0.07])  # [left, bottom, width, height]
    cbar_mu = fig.colorbar(im1, cax=cbar_mu_ax, orientation='horizontal')

    # Adjust layout for Mu and the colorbar
    plt.subplots_adjust(top=0.85, bottom=0.2, right=0.9 if J % 2 == 0 else 0.85)
    plt.suptitle('Mu', fontsize=40, y=0.95)
    plt.show()

    # Create subplots for Beta
    fig, axs_beta = plt.subplots(2, (J + 1) // 2, figsize=(21, 8.5))

     # Calculate global min and max for Mu
    global_min_beta = min([min(model.Beta[j](model.ones)) for j in range(J)])
    global_max_beta = max([max(model.Beta[j](model.ones)) for j in range(J)])


    # After training is complete
    for j, linear_layer in enumerate(model.Beta):
        # Pass ones vector through the linear layer, to see its values
        Beta_j = vec_2_blk_diag(model.Beta[j](model.ones))

        # Plot magnitude
        im1 = axs_beta[j // ((J + 1) // 2), j % ((J + 1) // 2)].imshow(Beta_j.squeeze(0).cpu().detach().numpy(), cmap='viridis', vmin=global_min_beta, vmax=global_max_beta)
        axs_beta[j // ((J + 1) // 2), j % ((J + 1) // 2)].set_title(f'Layer {j}')

    # Create a colorbar for Beta
    cbar_beta_ax = fig.add_axes([0.25, 0.08, 0.5, 0.07])  # [left, bottom, width, height]
    cbar_beta = fig.colorbar(im1, cax=cbar_beta_ax, orientation='horizontal')

    # Adjust layout for Beta and the colorbar
    plt.subplots_adjust(top=0.85, bottom=0.2, right=0.9 if J % 2 == 0 else 0.85)
    plt.suptitle('Beta', fontsize=40, y=0.95)
    plt.show()


def line_search(ls_iters):
    """ Iterates over every sample in the validation set, and finds the best step size for each iteration, for each sample. """
    # Initializing the PGA + M object with default values:
    ls_model = FastTrainUnfolding(W, J, "Constant")
    best_steps = torch.zeros((len(val_loader_sin)), ls_iters)
    best_Rs = torch.zeros((len(val_loader_sin)), ls_iters)
    grid = torch.cat((torch.linspace(0.01,0.5,6),torch.linspace(1,5,5)))
    # Takes a single sample:
    for idx, sample in enumerate(val_loader_sin):
        W_ls = W.detach().clone()
        sample.unsqueeze(0)
        # Start moving over all the iterations in "line_search_iters".
        for iteration in range(ls_iters):
            # initianting the best step and the best R to be 0:
            best_step = 0
            best_R = 0
            # Calculating the gradient:
            grad = ls_model.calc_dR_dW(sample, W_ls)
            # Looking for the best step size with the calculated gradient:
            for step in grid:
                # Updating W with the step size:
                W_temp = W_ls + step * grad
                if (iteration + 1) % project_every == 0:
                    W_temp = hard_mag_proj(W_temp,binary_thresh=0)
                    if W_is_finite:
                      W_temp = D * torch.exp(1j*hard_phase_proj(W_temp))

                # Calculating the optional R.
                R = ls_model.calc_R(sample, W_temp)
                if abs(R) > abs(best_R): # If the calculated R is better than the best R so far, save the step size.
                    best_R = R
                    best_step = step
            # Before the next iteration, update W with the best step chosen.
            W_ls = W_ls + best_step * grad
            if (iteration + 1) % project_every == 0:
                W_ls = hard_mag_proj(W_ls,binary_thresh= 0)
                if W_is_finite:
                    W_ls = D * torch.exp(1j*hard_phase_proj(W_ls))
            best_steps[idx,iteration] = best_step # saving the best step and the best R.
            best_Rs[idx,iteration] = best_R
    average_best_Rs = torch.mean(best_Rs, dim=0)  # Calculate the mean R
    average_best_steps = torch.mean(best_steps, dim=0)  # Calculate the mean best step size, for comparison reasons.
    return average_best_Rs / K, average_best_steps


def manifold_optimization(MO_iters):
    """ Iterates over every sample in the validation set, and finds the best step size for each iteration,
     following the objective of approching G_opt, achieved by SVD. """
    # Initializing the PGA + M object with defatult values:
    mo_model = FastTrainUnfolding(W, J, "Constant")
    best_steps = torch.zeros((len(val_loader_sin)), MO_iters)
    best_Rs = torch.zeros((len(val_loader_sin)), MO_iters)
    grid = torch.cat((torch.linspace(0.01,0.5,3),torch.linspace(1,5,3)))
    W_init = W.detach().clone()
    E = torch.zeros((M, T), dtype=torch.complex128, device = device)
    E[:T, :T] = torch.eye(T)
    # Takes a single sample:
    for idx, sample in enumerate(val_loader_sin):
        W_0 = W_init.detach().clone().unsqueeze(0)
        prev_grad = torch.zeros_like(W_init)
        # Start moving over all the iterations in "MO_iters" (Usually 50).
        for iteration in range(MO_iters):
            # initianting the best step and the best R to be 0:
            best_step = 0
            best_R = 0
            min_error = 10**8  # taking a high number, to be used as "inifinity" for initilizaing the algorithm

            # Calculates the gradient:
            H_H_herm = sample @ (sample.conj().transpose(2,3))
            eigenvalues, eigenvectors = eig(H_H_herm)
            # Arrange eigenvectors in descending order
            sorted_indices = torch.argsort(abs(eigenvalues), descending=True)
            F = torch.stack([(eigenvectors[0,b,:,:])[sorted_indices[0,b]] for b in range(B)])
            G_opt = F @ E
            G_opt = torch.mean(G_opt, dim=0)
            grad = -2*(G_opt - W_0 @ A) @ A.transpose(0,1)

            #  Momentum:
            if iteration > 1 :
                # Using Polak-Ribiere momentum factor
                beta = grad * (grad - prev_grad) / (torch.norm(prev_grad)**2 + 10**-6)
                grad += beta * prev_grad

            # Calculating the inner product between the grad and the conjugate matrix:
            inner_product = (torch.real(grad * W_0.conj()))
            # Looking for the best step size with the calculated gradient:
            for step in grid:
                # Updating W with the step size:
                # Calculating the inner product between the grad and the conjugate matrix
                W_temp = W_0 + step * (grad - inner_product * W_0)
                if (iteration + 1) % project_every == 0:
                    W_temp = hard_mag_proj(W_temp,binary_thresh=0)
                    if W_is_finite:
                      W_temp = D * torch.exp(1j*hard_phase_proj(W_temp))

                # Calculating the optional error.
                curr_error = torch.norm(G_opt - W_temp @ A)
                if abs(curr_error) < abs(min_error): # If the calculated R is better than the best R so far, save the step size.
                    best_R = mo_model.calc_R(sample, W_temp)
                    best_step = step
                    min_error = abs(curr_error)

            # Before the next iteration, update W with the best step chosen.
            W_0 = W_0 + best_step * (grad - inner_product * W_0)
            if (iteration + 1) % project_every == 0:
                W_0 = hard_mag_proj(W_0,binary_thresh=0)

            #saving previuos to calculate the next beta according to Polak Ribiere:
            prev_grad = grad
            best_steps[idx,iteration] = best_step # saving the best step and the best R.
            best_Rs[idx,iteration] = best_R

    average_best_Rs = torch.mean(best_Rs, dim=0)  # Calculate the mean R
    average_best_steps = torch.mean(best_steps, dim=0)  # Calculate the mean best step size, for comparison reasons.
    return average_best_Rs / K, average_best_steps
pass


##Running times: (On CPU)


Taking a single batch, of 10,000 samples.

##Scenario 2

In [ ]:
# # Setting parameters
W, B, K, P, N, M, L, T, snr, A, Ik, Im, D, J,\
epochs, W_is_block_diag, W_is_phase_only, batch_size, \
pga_iters, lr_mu_beta, lr_lambda, binary_thresh, train_size,\
valid_size, lambdas_init, gamma ,mat_file_path, project_every,\
l1_target, device, sharpness, Q, W_is_finite \
= set_params(batch_size = 10000, valid_size = 10000,
             P=4, L=2, N=3, K=5, B=4, T=5)

# # Generate the data
train_loader, val_loader, val_loader_sin = gen_data(train_size, valid_size, "I.I.D.")
u_model = FastTrainUnfolding(W,J,"Unfolding").to(device)
u_model.eval()

const_model = FastTrainUnfolding(W,J,"Constant", mu = torch.randn(1),beta = torch.randn(1)).to(device)
const_model.eval()

cnn_model = FastTrainUnfolding(W,J,"CNN", cnn_layers = 10).to(device)
cnn_model.eval()
pass

In [ ]:
with torch.no_grad():

    init_time = time.time()
    _ = cnn_model.forward_cnn(next(iter(val_loader)))
    final_time = time.time()
    print('CNN 10 layers inference time: ', round(10**6*(final_time - init_time) / valid_size), 'us')


    init_time = time.time()
    _ = u_model.forward_unfolded(next(iter(val_loader)))
    final_time = time.time()
    print('U - PGA + M  10 iterations time: ', round(10**6*(final_time - init_time) / valid_size), 'us')

    const_model.pga_iters = 10
    train_loader, val_loader, _ = gen_data(train_size, valid_size, "I.I.D.")
    init_time = time.time()
    _ = const_model.forward_constant(next(iter(val_loader)))
    final_time = time.time()
    print('PGA + M  10 iterations time: ', round(10**6*(final_time - init_time)/valid_size), 'us')

    const_model.pga_iters = 500
    train_loader, val_loader, _ = gen_data(train_size, valid_size, "I.I.D.")
    init_time = time.time()
    _ = const_model.forward_constant(next(iter(val_loader)))
    final_time = time.time()
    print('PGA + M  500 iterations time: ', round(10**6*(final_time - init_time)/valid_size), 'us')


    init_time = time.time()
    _ = line_search(ls_iters = 10)
    final_time = time.time()
    print('Line Search 10 iterations time: ', round(10**6*(final_time - init_time)/valid_size), 'us')


    init_time = time.time()
    _ = manifold_optimization(MO_iters = 10)
    final_time = time.time()
    print('Manifold Optimization 10 iterations time: ', round(10**6*(final_time - init_time)/valid_size), 'us')



CNN 10 layers inference time:  290 us
U - PGA + M  10 iterations time:  417 us
PGA + M  10 iterations time:  413 us
PGA + M  500 iterations time:  19352 us
Line Search 10 iterations time:  123705 us
Manifold Optimization 10 iterations time:  31767 us
